# Update Multi-Task Superconductor Generator from GitHub

Run this notebook to pull the latest code from GitHub into your Google Drive repo.
Preserves checkpoints and training logs. Invalidates tensor cache if data changed.

**Keep this notebook on Drive** — reuse it anytime you need to update.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# === CONFIGURATION ===
# Update this path if your repo is in a different Drive location
REPO_PATH = "/content/drive/My Drive/Colab Notebooks/SuperconductorVAE/superconductor-vae"
GITHUB_URL = "https://github.com/jamesconde/superconductor-vae.git"
BRANCH = "main"

In [ ]:
import os
import shutil
from pathlib import Path

repo = Path(REPO_PATH)
is_git_repo = (repo / '.git').exists()

if is_git_repo:
    # --- Fast path: git pull ---
    print("Git repo detected — pulling latest changes...")
    !cd "{REPO_PATH}" && git fetch origin {BRANCH} && git reset --hard origin/{BRANCH}
    print("Done!")

else:
    # --- First time or tarball extraction: clone fresh, preserve outputs ---
    print("No .git directory — cloning fresh from GitHub...")
    print("Preserving outputs/ (checkpoints, logs, norm_stats)...")

    # Save outputs we want to keep
    preserve_dir = Path("/content/_preserved_outputs")
    preserve_dir.mkdir(exist_ok=True)
    outputs_dir = repo / "outputs"

    preserved_files = []
    if outputs_dir.exists():
        for f in outputs_dir.iterdir():
            if f.is_file() and f.suffix in ('.pt', '.json', '.csv', '.log'):
                dest = preserve_dir / f.name
                print(f"  Preserving: {f.name} ({f.stat().st_size / 1e6:.1f} MB)")
                shutil.copy2(str(f), str(dest))
                preserved_files.append(f.name)

    # Also preserve tensor cache if it exists
    cache_dir = repo / "data" / "processed" / "cache"
    preserved_cache = False
    if cache_dir.exists():
        print(f"  Preserving tensor cache...")
        shutil.copytree(str(cache_dir), "/content/_preserved_cache", dirs_exist_ok=True)
        preserved_cache = True

    # Remove old repo and clone
    if repo.exists():
        shutil.rmtree(str(repo))
    !git clone --branch {BRANCH} {GITHUB_URL} "{REPO_PATH}"

    # Restore outputs
    (repo / "outputs").mkdir(exist_ok=True)
    for fname in preserved_files:
        src = preserve_dir / fname
        dest = repo / "outputs" / fname
        shutil.copy2(str(src), str(dest))
        print(f"  Restored: {fname}")

    # Restore cache
    if preserved_cache:
        cache_dest = repo / "data" / "processed" / "cache"
        cache_dest.mkdir(parents=True, exist_ok=True)
        shutil.copytree("/content/_preserved_cache", str(cache_dest), dirs_exist_ok=True)
        print("  Restored tensor cache")

    # Cleanup temp
    shutil.rmtree(str(preserve_dir), ignore_errors=True)
    shutil.rmtree("/content/_preserved_cache", ignore_errors=True)

    print(f"\nDone! Repo is now a git clone at: {REPO_PATH}")
    print("Future updates will use fast 'git pull'.")

In [ ]:
# === Optional: Invalidate tensor cache ===
# Run this cell if the dataset CSV or preprocessing config changed.
# Training will rebuild the cache on next run (~2 min).

cache_meta = Path(REPO_PATH) / "data/processed/cache/cache_meta.json"
if cache_meta.exists():
    cache_meta.unlink()
    print("Tensor cache invalidated — will rebuild on next training run.")
else:
    print("No cache to invalidate (already fresh or doesn't exist).")

## V12.28: Post-Update Migration Steps

If this is the **first update after V12.28**, run the cells below to:
1. Install new dependencies (matminer)
2. Ingest JARVIS superconductor data (adds ~4,300 new samples)
3. Migrate the checkpoint to the new architecture (Net2Net weight transfer)

**These only need to run ONCE.** After that, skip to the Verify cell.

In [ ]:
# === V12.28 Step 1: Install matminer (needed for JARVIS ingestion) ===
# matminer computes 146 material features (Magpie, valence, ion properties, etc.)
# Skip if already installed.

try:
    import matminer
    print(f"matminer already installed: v{matminer.__version__}")
except ImportError:
    print("Installing matminer...")
    !pip install matminer -q
    print("Done!")

In [ ]:
# === V12.28 Step 2: Ingest JARVIS superconductor data ===
# Adds ~4,300 new samples from JARVIS databases + 6 physics feature columns.
# Safe to re-run (deduplicates against existing data).
# Takes ~2 minutes (matminer feature computation is the bottleneck).

import subprocess, sys

csv_path = Path(REPO_PATH) / "data/processed/supercon_fractions_contrastive.csv"

# Check if JARVIS data was already ingested (>50K rows means it's done)
if csv_path.exists():
    import pandas as pd
    n_rows = len(pd.read_csv(csv_path, usecols=[0]))
    if n_rows > 50000:
        print(f"JARVIS data already ingested ({n_rows} rows). Skipping.")
    else:
        print(f"Current CSV has {n_rows} rows. Running JARVIS ingestion...")
        !cd "{REPO_PATH}" && PYTHONPATH=src python scripts/ingest_jarvis.py
        print("\nDone! Re-check row count:")
        n_rows_new = len(pd.read_csv(csv_path, usecols=[0]))
        print(f"  {n_rows} → {n_rows_new} rows")
else:
    print("WARNING: CSV not found. Run training once first to generate the base CSV,"
          " then re-run this cell.")

In [ ]:
# === V12.28 Step 3: Migrate checkpoint to new architecture ===
# Applies Net2Net weight transfer from old Tc head to new residual Tc head.
# Backs up the original checkpoint before modifying it.
# Safe to re-run (skips if already migrated).

ckpt_path = Path(REPO_PATH) / "outputs/checkpoint_best.pt"
backup_path = ckpt_path.with_suffix('.pt.bak_pre_v1228')

if not ckpt_path.exists():
    print("No checkpoint found — nothing to migrate. Training will start fresh.")
elif backup_path.exists():
    print(f"Checkpoint already migrated (backup exists at {backup_path.name}). Skipping.")
else:
    # Dry run first to verify
    print("Running migration dry-run...")
    !cd "{REPO_PATH}" && PYTHONPATH=src python scripts/migrate_checkpoint_v1228.py --dry-run
    print("\n" + "="*50)
    print("Dry run OK. Now applying migration for real...")
    print("="*50 + "\n")
    !cd "{REPO_PATH}" && PYTHONPATH=src python scripts/migrate_checkpoint_v1228.py
    print("\nCheckpoint migrated! Original backed up as checkpoint_best.pt.bak_pre_v1228")

In [ ]:
# === V12.28 Step 4: Invalidate tensor cache (dataset changed) ===
# The CSV now has new rows and columns — old tensor cache is stale.
# Training will rebuild it automatically on next run (~2 min).

cache_meta = Path(REPO_PATH) / "data/processed/cache/cache_meta.json"
cache_dir = Path(REPO_PATH) / "data/processed/cache"

if cache_dir.exists() and any(cache_dir.iterdir()):
    shutil.rmtree(str(cache_dir))
    cache_dir.mkdir(exist_ok=True)
    print("Tensor cache cleared — will rebuild on next training run.")
else:
    print("No tensor cache to clear.")

In [ ]:
# === Verify ===
print("Repo contents:")
!ls "{REPO_PATH}/"
print()
print("Latest commit:")
!cd "{REPO_PATH}" && git log --oneline -3
print()
print("Outputs:")
!ls -lh "{REPO_PATH}/outputs/" 2>/dev/null | head -10
print()
print("Now close this notebook and open train_colab.ipynb to start training.")